# **Penting**
- Pastikan Anda melakukan Run All sebelum mengirimkan submission untuk memastikan seluruh cell berjalan dengan baik.
- Hapus simbol pagar (#) jika Anda menerapkan kriteria tambahan
- Biarkan simbol pagar (#) jika Anda tidak menerapkan kriteria tambahan

# **1. Import Library**
Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [96]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


# **2. Memuat Dataset dari Hasil Clustering**
Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [97]:
# Gunakan dataset hasil clustering yang memiliki fitur Target
# Silakan gunakan dataset data_clustering jika tidak menerapkan Interpretasi Hasil Clustering [Advanced]
# Silakan gunakan dataset data_clustering_inverse jika menerapkan Interpretasi Hasil Clustering [Advanced]


# Ganti dengan URL mentah dari file di GitHub kamu (Raw file URL)
url = 'https://raw.githubusercontent.com/AldiPrasetyoHari/BMLP_Dicoding/main/data_clustering_inverse.csv'

# Membaca dataset
df = pd.read_csv(url)


In [98]:
# Tampilkan 5 baris pertama dengan function head.
df.head(5)

,TransactionAmount,CustomerAge,TransactionDuration,LoginAttempts,AccountBalance,TransactionDate,PreviousTransactionDate,is_weekend,TransactionType,Location,Channel,CustomerOccupation,Target
0,14.09,70.0,81.0,1.0,5112.21,2023-04-11 16:29:14,2024-11-04 08:08:08,WeekDays,Debit,San Diego,ATM,Doctor,2
1,376.24,68.0,141.0,1.0,13758.91,2023-06-27 16:44:19,2024-11-04 08:09:35,WeekDays,Debit,Houston,ATM,Doctor,2
2,126.29,19.0,56.0,1.0,1122.35,2023-07-10 18:16:08,2024-11-04 08:07:04,WeekDays,Debit,Mesa,Online,Student,2
3,184.50,26.0,25.0,1.0,8569.06,2023-05-05 16:32:11,2024-11-04 08:09:06,WeekEnds,Debit,Raleigh,Online,Student,0
4,13.45,45.0,198.0,1.0,7429.40,2023-10-16 17:51:24,2024-11-04 08:06:39,WeekDays,Credit,Atlanta,Online,Student,1


Tabel tersebut menampilkan cuplikan data yang diperoleh dari hasil inverse clustering sebelumnya dimana sudah terdapat juga fitur Target yang merupakan hasil dari clustering segmentasi customer.

In [99]:
kolnum = ['TransactionAmount', 'CustomerAge', 'TransactionDuration', 'LoginAttempts', 'AccountBalance']

scaler = MinMaxScaler()

df_scaled = df.copy()

df_scaled[kolnum] = scaler.fit_transform(df_scaled[kolnum])

categorical_cols = ['is_weekend','TransactionType', 'Location', 'Channel', 'CustomerOccupation']

df_encoded = pd.get_dummies(df_scaled, columns=categorical_cols, drop_first=False).replace({True: 1, False: 0})

# df_encoded.info()
df_encoded.to_csv('df_encoded.csv', index=False)


<ipython-input-99-232604771120>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_encoded = pd.get_dummies(df_scaled, columns=categorical_cols, drop_first=False).replace({True: 1, False: 0})


Oleh karena menggunakan data yang sudah di Inverse dari tahap clustering sebelum nya, maka perlu dilakukan preprocessing kembali agar setiap fitur berubah dalam bentuk numerik.

# **3. Data Splitting**
Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [100]:
# Menggunakan train_test_split() untuk melakukan pembagian dataset.
X = df_encoded.drop(columns=['Target','TransactionDate','PreviousTransactionDate'])
y = df_encoded['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# **4. Membangun Model Klasifikasi**
Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Menggunakan algoritma klasifikasi yaitu Decision Tree.
2. Latih model menggunakan data yang sudah dipisah.

In [101]:
# Buatlah model klasifikasi menggunakan Decision Tree

dt = DecisionTreeClassifier().fit(X_train, y_train)

In [102]:
# Menyimpan Model
joblib.dump(dt, 'decision_tree_model.h5')

['decision_tree_model.h5']

Model Decision Tree telah di latih dan disimpan.

# **5. Memenuhi Kriteria Skilled dan Advanced dalam Membangun Model Klasifikasi**



**Biarkan kosong jika tidak menerapkan kriteria skilled atau advanced**

In [103]:
# Melatih model menggunakan algoritma klasifikasi selain Decision Tree.
knn = KNeighborsClassifier().fit(X_train, y_train)
rf = RandomForestClassifier().fit(X_train, y_train)
svm = SVC().fit(X_train, y_train)

In [104]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada seluruh algoritma yang sudah dibuat.
def cek_performa_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    metrics = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='weighted'),
        'Recall': recall_score(y_test, y_pred, average='weighted'),
        'F1-Score': f1_score(y_test, y_pred, average='weighted')
    }
    return metrics
model_performance = {
    'K-Nearest Neighbors': cek_performa_model(knn, X_test, y_test),
    'Decision Tree': cek_performa_model(dt, X_test, y_test),
    'Random Forest': cek_performa_model(rf, X_test, y_test),
    'Support Vector Machine': cek_performa_model(svm, X_test, y_test)
}
param_eval = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])
model_eval = []
for name, result in model_performance.items():
    model_eval.append({
        'Model': name,
        'Accuracy': result['Accuracy'],
        'Precision': result['Precision'],
        'Recall': result['Recall'],
        'F1-Score': result['F1-Score']

    })
param_eval = pd.DataFrame(model_eval)
KNNBeforeTunning=param_eval[param_eval['Model'] == 'K-Nearest Neighbors'].copy()
KNNBeforeTunning['Category']='Sebelum Tunning'
param_eval.sort_values(by='Accuracy', ascending=False)
# param_eval


,Model,Accuracy,Precision,Recall,F1-Score
1,Decision Tree,1.000000,1.000000,1.000000,1.000000
2,Random Forest,0.995943,0.995943,0.995943,0.995943
3,Support Vector Machine,0.853955,0.865458,0.853955,0.855753
0,K-Nearest Neighbors,0.772819,0.781108,0.772819,0.775356


Bedasarkan data tersebut dapat dilihat model DecisionTree memiliki nilai parameter evaluasi terbaik yang bernilai 1. Namun ini juga dapat menjadi indikasi bahwa model terlalu menghafal data pelatihan yang dapat mengakibatkan overfit. Sedangkan model dengan nilai evaluasi parameter terendah terdapat pada model KNN, Oleh karena itu akan dilakukan percobaan tuning terhadap model tersebut untuk dapat melihat performa evaluasi model KNN dapat lebih baik atau tidak.

In [105]:
# Menyimpan Model Selain Decision Tree
# Model ini bisa lebih dari satu
joblib.dump(rf, 'explore_randomforest_classification.h5')
joblib.dump(svm, 'explore_svm_classification.h5')

['explore_svm_classification.h5']

Model Random Forest dan SVM memberikan metrik evaluasi yang cukup baik dibandingkan dengan K-Nearest Neighbors.

Hyperparameter Tuning Model

Pilih salah satu algoritma yang ingin Anda tuning

In [106]:
# Lakukan Hyperparameter Tuning dan Latih ulang.
knn_params = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

models = {
    'KNN': (KNeighborsClassifier(), knn_params),
}

# best_models = {}
# for name, (model, params) in models.items():

# grid = GridSearchCV(model, params, cv=5, scoring='f1_weighted', n_jobs=-1)
grid = GridSearchCV(KNeighborsClassifier(), params, cv=5, scoring='f1_weighted', n_jobs=-1)
knnTunned=grid.fit(X_train, y_train)
print(knnTunned.best_estimator_)
# best_models[name] = grid.best_estimator_
print(f"KNN Params : {knnTunned.best_params_}")

KNeighborsClassifier(metric='manhattan', n_neighbors=7)
KNN Params : {'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'uniform'}


Dengan menggunakan metode gridSearchCV setiap nilai rentang yang digunakan pada hyperparameter akan di uji dan di evaluasi hingga diperoleh nilai hyperparameter yang optimal yang dapat digunakan untuk meningkatkan performa dari model K-Nearest Neighbor yang sebelum di tunning memiliki nilai yang paling kecil dalam metrik evaluasi.

In [107]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada algoritma yang sudah dituning.
model_summaries = []

result = cek_performa_model(knnTunned.best_estimator_, X_test, y_test)
model_summaries.append({
  'Category': 'Setelah Tunning',
  'Model': 'K-Nearest Neighbors',
  'Accuracy': result['Accuracy'],
  'Precision': result['Precision'],
  'Recall': result['Recall'],
  'F1-Score': result['F1-Score']
})

KNNAfterTunning=pd.DataFrame(model_summaries)
df_knn_comparison = pd.concat([KNNBeforeTunning, KNNAfterTunning], ignore_index=True)
df_knn_comparison

,Model,Accuracy,Precision,Recall,F1-Score,Category
0,K-Nearest Neighbors,0.772819,0.781108,0.772819,0.775356,Sebelum Tunning
1,K-Nearest Neighbors,0.811359,0.817368,0.811359,0.813134,Setelah Tunning


Berdasarkan hasil tunning model K-Nearest Neighbor yang telah dilakukan dapat diketahui jika metrik evaluasi yang diperoleh memiliki nilai evaluasi yang lebih baik dibandingkan sebelum melakukan tunning. Hal ini mengindikasikan jika tuning hyperparameter telah sesuai atau sudah optimal dengan menggunakan metode gridSearchCV. Hyperparametert pada K-Nearest Neighbor yang ditunning yakni n_neighbors, weights, dan metric.

In [108]:
# Menyimpan Model hasil tuning
joblib.dump(knnTunned.best_estimator_, 'tuning_classification.h5')

['tuning_classification.h5']